Импорт основных библиотек

In [1]:
import pandas as pd
%matplotlib inline
import requests
import bs4
import pprint

# 1. Задача: Извлечь данные по таблицам Бомбардиры, Ассистенты, Штрафники.

In [2]:
url="https://soccer365.ru/competitions/13"
tables = pd.read_html(url)

# Бомбардиры:

In [3]:
display(tables[1])

,Бомбардиры,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Владимир Сычевой,14,2.0,15
1,Квинси Промес,14,2.0,16
2,Малком,12,NaN,17
3,Кристиан Нобоа,10,6.0,15
4,Федор Чалов,10,2.0,17
5,Эдуард Сперцян,9,3.0,16
6,Дмитрий Полоз,8,6.0,16
7,Николай Комличенко,8,1.0,17
8,Мохамед Конате,7,NaN,16
9,Тимур Сулейманов,7,1.0,16


# Ассистенты:

In [4]:
display(tables[2])

,Ассистенты,Unnamed: 1,Unnamed: 2
0,Александр Соболев,6,16
1,Эдуард Сперцян,6,16
2,Квинси Промес,6,16
3,Даниил Уткин,6,17
4,Клаудиньо,5,14
5,Арсен Захарян,5,16
6,Иван Олейников,5,17
7,Николай Комличенко,5,17
8,Малком,5,17
9,Никита Бурмистров,4,9


# Штрафники:

In [5]:
display(tables[3])

,Штрафники,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Роберто Фернандес,9,3,2.0,NaN,11
1,Мамаду Майга,9,6,1.0,NaN,14
2,Антон Швец,7,7,NaN,NaN,16
3,Андрей Егорычев,6,3,1.0,NaN,9
4,Кайо,6,3,1.0,NaN,11
5,Мингиян Бевеев,6,3,1.0,NaN,14
6,Артём Самсонов,6,6,NaN,NaN,15
7,Олег Дмитриев,6,3,NaN,1.0,16
8,Тимур Сулейманов,6,3,1.0,NaN,16
9,Александр Соболев,6,6,NaN,NaN,16


# 2. Собрать из этих данных датафрейм.

Парсинг команд для каждой из таблиц Штрафники, Бомбардиры и Ассистенты

In [6]:
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
player_tables = soup.find_all('table', class_='comp_table_v2')

scorers = []
assistants = []
penalized_players = []
lists = (scorers, assistants, penalized_players)
for table, lst in zip(player_tables, lists):
    teams = table.find_all('img', class_='i16 has-tip')
    for team in teams:
        lst.append(team.get('title'))

titles = ('Бомбардиры:', 'Ассистенты:', 'Штрафники:')
for title, lst in zip(titles, lists):
    print(title, lst)

Бомбардиры: ['Оренбург', 'Спартак', 'Зенит', 'ПФК Сочи', 'ЦСКА Москва', 'Краснодар', 'Ростов', 'Ростов', 'Ахмат', 'Пари Нижний Новгород', 'Динамо Москва', 'Спартак']
Ассистенты: ['Спартак', 'Краснодар', 'Спартак', 'Ростов', 'Зенит', 'Динамо Москва', 'Ахмат', 'Ростов', 'Зенит', 'ПФК Сочи', 'Урал', 'Оренбург']
Штрафники: ['Динамо Москва', 'Пари Нижний Новгород', 'Ахмат', 'Урал', 'Краснодар', 'Урал', 'Торпедо Москва', 'Факел', 'Пари Нижний Новгород', 'Спартак', 'Динамо Москва', 'Локомотив Москва']


In [7]:
table_bomb = tables[1]
table_bomb.rename(columns = {'Бомбардиры':'ФИО', 'Unnamed: 1':'Голы','Unnamed: 2':'Пенальти', 'Unnamed: 3':'Матчи'}, inplace=True)
table_bomb.insert(1, "Команда", scorers)
table_bomb = table_bomb.fillna(0)
table_bomb['Пенальти'] = table_bomb['Пенальти'].astype('int')
display(table_bomb)

,ФИО,Команда,Голы,Пенальти,Матчи
0,Владимир Сычевой,Оренбург,14,2,15
1,Квинси Промес,Спартак,14,2,16
2,Малком,Зенит,12,0,17
3,Кристиан Нобоа,ПФК Сочи,10,6,15
4,Федор Чалов,ЦСКА Москва,10,2,17
5,Эдуард Сперцян,Краснодар,9,3,16
6,Дмитрий Полоз,Ростов,8,6,16
7,Николай Комличенко,Ростов,8,1,17
8,Мохамед Конате,Ахмат,7,0,16
9,Тимур Сулейманов,Пари Нижний Новгород,7,1,16


In [8]:
table_assist = tables[2]
table_assist.rename(columns = {'Ассистенты':'ФИО', 'Unnamed: 1':'Пасы','Unnamed: 2':'Матчи'}, inplace=True)
table_assist.insert(1, "Команда", assistants)

display(table_assist)

,ФИО,Команда,Пасы,Матчи
0,Александр Соболев,Спартак,6,16
1,Эдуард Сперцян,Краснодар,6,16
2,Квинси Промес,Спартак,6,16
3,Даниил Уткин,Ростов,6,17
4,Клаудиньо,Зенит,5,14
5,Арсен Захарян,Динамо Москва,5,16
6,Иван Олейников,Ахмат,5,17
7,Николай Комличенко,Ростов,5,17
8,Малком,Зенит,5,17
9,Никита Бурмистров,ПФК Сочи,4,9


In [9]:
table_shtraf = tables[3]
table_shtraf.rename(columns = {'Штрафники':'ФИО', 'Unnamed: 1':'Штрафные','Unnamed: 2':'Желтые карточки','Unnamed: 3':'2ЖК','Unnamed: 4':'Красные карточки','Unnamed: 5':'Матчи'}, inplace=True)
table_shtraf.insert(1, "Команда", penalized_players)
table_shtraf = table_shtraf.fillna(0)
table_shtraf['2ЖК'] = table_shtraf['2ЖК'].astype('int')
table_shtraf['Красные карточки'] = table_shtraf['Красные карточки'].astype('int')
display(table_shtraf)

,ФИО,Команда,Штрафные,Желтые карточки,2ЖК,Красные карточки,Матчи
0,Роберто Фернандес,Динамо Москва,9,3,2,0,11
1,Мамаду Майга,Пари Нижний Новгород,9,6,1,0,14
2,Антон Швец,Ахмат,7,7,0,0,16
3,Андрей Егорычев,Урал,6,3,1,0,9
4,Кайо,Краснодар,6,3,1,0,11
5,Мингиян Бевеев,Урал,6,3,1,0,14
6,Артём Самсонов,Торпедо Москва,6,6,0,0,15
7,Олег Дмитриев,Факел,6,3,0,1,16
8,Тимур Сулейманов,Пари Нижний Новгород,6,3,1,0,16
9,Александр Соболев,Спартак,6,6,0,0,16


# 3 Получить по датафрейму следующую аналитику:

3.1 Первая тройка команд по числу забитых голов с выводом их числа.

In [10]:
table_bomb.groupby(["Команда"])["Голы"].sum().sort_values(ascending=False).head(3)

Команда
Спартак     21
Ростов      16
Оренбург    14
Name: Голы, dtype: int64

3.2 Первая тройка команд по числу желтых карточек.

In [11]:
table_shtraf.groupby(["Команда"])["Желтые карточки"].sum().sort_values(ascending=False).head(3)

Команда
Пари Нижний Новгород    9
Ахмат                   7
Динамо Москва           6
Name: Желтые карточки, dtype: int64

3.3 Доля пенальти по отношению к числу голов для каждой команды.

In [12]:
count_penalti =table_bomb.groupby(["Команда"])["Пенальти"].sum()
count_gol = table_bomb.groupby(["Команда"])["Голы"].sum()
(count_penalti/count_gol)

Команда
Ахмат                   0.000000
Динамо Москва           0.428571
Зенит                   0.000000
Краснодар               0.333333
Оренбург                0.142857
ПФК Сочи                0.600000
Пари Нижний Новгород    0.142857
Ростов                  0.437500
Спартак                 0.095238
ЦСКА Москва             0.200000
dtype: float64

3.4 Список игроков, которые участвовали не во всех играх своей команды. Число игр команды определить по максимальному числу матчей ее игроков.

In [13]:
join_table = table_bomb.append([table_assist,table_shtraf], sort=False)#соединил все таблицы

count_plays_team = join_table.groupby(['Команда']).max(['Матчи']) #число игр команды
gamers = []

for num_row in range(len(join_table)):
  player_name = join_table.iloc[num_row,0]
  player_count_game = join_table.iloc[num_row,4]

  player_team = join_table.iloc[num_row,1]

  team_max_games = count_plays_team.loc[player_team]['Матчи']
  if int(player_count_game) != int(team_max_games):
    gamers.append(player_name+f" сыграл {player_count_game} из {int(team_max_games)}")
pprint.pprint(gamers)

['Дмитрий Полоз сыграл 16 из 17',
 'Мохамед Конате сыграл 16 из 17',
 'Фёдор Смолов сыграл 15 из 16',
 'Клаудиньо сыграл 14 из 17',
 'Никита Бурмистров сыграл 9 из 15',
 'Лазарь Ранджелович сыграл 9 из 14',
 'Брайан Мансилья сыграл 12 из 15',
 'Роберто Фернандес сыграл 11 из 16',
 'Мамаду Майга сыграл 14 из 16',
 'Антон Швец сыграл 16 из 17',
 'Андрей Егорычев сыграл 9 из 14',
 'Кайо сыграл 11 из 16']


C:\Users\erik_\AppData\Local\Temp\ipykernel_10856\2358773928.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join_table = table_bomb.append([table_assist,table_shtraf], sort=False)#соединил все таблицы


3.5 Корреляция числа голов с количеством очков команды. Очки взять из первой таб-лицы на странице по рейтингу команд.

In [14]:
table_ochki = tables[0]

table_ochki.rename(columns = {'Unnamed: 0':'Рейтинг', 'Unnamed: 1':'Команда','З':'Забито', 'О':'Очки'}, inplace=True)

a = table_ochki.groupby(["Команда"])[["Забито"]].sum()
b = table_ochki.groupby(["Команда"])[["Очки"]].sum()
c = b.join(a)
c = c.fillna(0)
c = c.reset_index()
c.corr()

C:\Users\erik_\AppData\Local\Temp\ipykernel_10856\1975397723.py:10: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  c.corr()


,Очки,Забито
Очки,1.000000,0.947128
Забито,0.947128,1.000000
